# 9E_vocab_compare_gene

6/17/21

Can you see if there is any target gene overlap between any of the 'AND' vocabs for MM vs MC and for SCC vs KCD6? Maybe can scrounge up a  v4C vignette showing how cancer vs normal use different TFs to regulate the same genes? Engh?

And then if there ARE any overlapping target genes, would also need the genomic locations of the vocabs




In [80]:
import pybedtools
import glob, os
import pandas as pd
from collections import defaultdict

In [2]:
bed_file = '../data/interim/merged/atac/Colon_merged.bed'

# pandas
# pd.read_table(bed_file, names = ['chr','start','stop','name','chrx','startx','stopy','rowname','distance'])

pybedtools.BedTool(bed_file).merge().sort().to_dataframe()



,chrom,start,end
0,chr1,713908,714326
1,chr1,762050,763512
2,chr1,781110,781315
3,chr1,793350,794448
4,chr1,840069,840282
...,...,...,...
67456,chrY,21154493,21155089
67457,chrY,23153060,23153234
67458,chrY,26416783,26417235
67459,chrY,27545204,27545651


In [3]:
save_dir = '../data/processed/fig7_vocab_go/'
vocab_geneset_dir = '../data/processed/fig7_vocab_go/vocab_geneset_all_tissue'


In [4]:
# tfs 
tf_annon_df = pd.read_csv('../data/external/HOCOMOCOv11_annotation.csv',index_col=0)
tf_annon_df['id_trim'] = tf_annon_df['id'] + '.pwm.trim'
tf_name_to_id_dict = pd.Series(tf_annon_df.id_trim.values, index=tf_annon_df.tf.values).to_dict()
tf_id_to_name_dict = pd.Series(tf_annon_df.tf.values, index=tf_annon_df.id_trim.values).to_dict()

# 1. cancer+normal using the same vocab for the same target gene?



In [5]:
# get mpra vocabs + configs
new_mpra_df = pd.read_csv('../data/processed/mpra/results_df_piv_match_count.csv',index_col=0)
# filter for skin stuff
new_mpra_df = new_mpra_df[new_mpra_df.tissue.isin(['GDSD6','SCC','Melanocytes','MM'])]
# filter for AND VOCABs
new_mpra_df = new_mpra_df[new_mpra_df.consensus_class=='AND']
print(new_mpra_df.shape)

# new_mpra_df = new_mpra_df[new_mpra_df.tissue.isin(['MM','SCC'])][['vocab','tissue']].rename(columns={"vocab": "vocab_pair"}).drop_duplicates()
new_mpra_df.columns

(35, 9)


Index(['vocab', 'tissue', 'mpra_tissue', 'class', 'instance', 'count',
       'consensus_class', 'tf1', 'tf2'],
      dtype='object')

In [6]:
new_mpra_df['tissues_to_check'] = ''
new_mpra_df['num_overlap_genes'] = 0
new_mpra_df['num_normal_genes'] = 0
new_mpra_df['num_cancer_genes'] = 0
new_mpra_df['overlap_genes'] = ''
new_mpra_df['normal_genes'] = ''
new_mpra_df['cancer_genes'] = ''
# new_mpra_df['Melanocytes_genes'] = ''
# new_mpra_df['MM_genes'] = ''
for idx, row in new_mpra_df.iterrows():
    print(row['vocab'],row.tissue)
    tf1,tf2 = row['vocab'].split('::')
    
    # get tissues to check
    if row.tissue in ['GDSD6',  'SCC']:
        tissues_to_check = ['GDSD6',  'SCC']
    else:
        tissues_to_check = ['Melanocytes','MM']
    new_mpra_df.at[idx, 'tissues_to_check'] = '|'.join(tissues_to_check )
    
    genesets = [set(),set()]
    for i_mpra,mpra_tissue in enumerate(tissues_to_check):
        geneset_file = os.path.join(vocab_geneset_dir,row['tissue']+'_'+tf1 + '-' + tf2+'_'+mpra_tissue+'.txt' )
        if os.path.exists(geneset_file):
            genesets[i_mpra] = set(pd.read_csv(geneset_file,header=None).iloc[:,0].values)
            
        
    print('num normal genes',len(genesets[0]))
    print('num cancer genes',len(genesets[1]))

    intersect_genes = genesets[0].intersection(genesets[1])
#     if len(intersect_genes)==0:
#         print('no overlapping genes')
#         continue
    print('num intersect genes',len(intersect_genes))

    new_mpra_df.at[idx, 'num_overlap_genes'] = len(intersect_genes)
    new_mpra_df.at[idx, 'num_normal_genes'] = len(genesets[0])
    new_mpra_df.at[idx, 'num_cancer_genes'] = len(genesets[1])
    new_mpra_df.at[idx, 'overlap_genes'] = '|'.join(sorted(intersect_genes))
    new_mpra_df.at[idx, 'normal_genes'] = '|'.join(sorted(genesets[0]))
    new_mpra_df.at[idx, 'cancer_genes'] = '|'.join(sorted(genesets[1]))
    print(idx)
#     new_mpra_df.at[idx, tissues_to_check[0]+'_genes'] = '|'.join(sorted(genesets[0]))
#     new_mpra_df.at[idx, tissues_to_check[1]+'_genes'] = '|'.join(sorted(genesets[1]))
   

ARID5B::LEF1 MM
num normal genes 13
num cancer genes 464
num intersect genes 10
1
ARNT::SP1 SCC
num normal genes 91
num cancer genes 828
num intersect genes 71
5
BACH1::KLF5 GDSD6
num normal genes 100
num cancer genes 988
num intersect genes 80
8
E2F6::SP1 SCC
num normal genes 274
num cancer genes 1385
num intersect genes 240
18
EGR1::SP3 SCC
num normal genes 327
num cancer genes 1432
num intersect genes 281
25
EGR2::FOSL2 GDSD6
num normal genes 255
num cancer genes 1164
num intersect genes 202
26
EGR2::NFE2L1 MM
num normal genes 0
num cancer genes 610
num intersect genes 0
31
EGR2::RARG GDSD6
num normal genes 196
num cancer genes 1103
num intersect genes 155
32
EGR2::RORA GDSD6
num normal genes 205
num cancer genes 0
num intersect genes 0
33
EN2::STAT1 MM
num normal genes 3
num cancer genes 459
num intersect genes 2
43
ETV2::PRRX1 Melanocytes
num normal genes 45
num cancer genes 553
num intersect genes 33
47
FOXD3::PRRX1 Melanocytes
num normal genes 21
num cancer genes 366
num interse

In [7]:
new_mpra_df[:5]

,vocab,tissue,mpra_tissue,class,instance,count,consensus_class,tf1,tf2,tissues_to_check,num_overlap_genes,num_normal_genes,num_cancer_genes,overlap_genes,normal_genes,cancer_genes
1,ARID5B::LEF1,MM,COLO,other|LEADING_B|other|OR|OR|AND|other|other|AN...,1|10|2|3|4|5|6|7|8|9,10,AND,ARID5B,LEF1,Melanocytes|MM,10,13,464,CLP1|DUSP7|FTL|MRPL10|MRPL34|N4BP1|OCEL1|PKD1|...,CLP1|CORO1C|DUSP7|FTL|MRPL10|MRPL34|N4BP1|NFE2...,ABCC5|ABCF3|ABRACL|ABTB2|ACADS|ACOT9|ACTG1|ACT...
5,ARNT::SP1,SCC,A431,AND|AND|OR|LEADING_A|AND|LEADING_A|LEADING_B|o...,1|10|2|3|4|5|6|7|8|9,10,AND,ARNT,SP1,GDSD6|SCC,71,91,828,AHNAK|AP1S1|AP3D1|ARHGEF3|ARID3B|CA11|CCDC51|C...,AHNAK|ALOX15B|AP1S1|AP3D1|ARHGEF3|ARID3B|CA11|...,AAMP|AARS2|ABCA7|ABCB6|ABCB8|ABCF3|ABHD10|ABTB...
8,BACH1::KLF5,GDSD6,KCD6,AND|LEADING_B|OR|other|OR|other|other|LEADING_...,1|10|2|3|4|5|6|7|8|9,10,AND,BACH1,KLF5,GDSD6|SCC,80,100,988,ABCB8|AHNAK2|ANKRD11|AP3D1|BCL2L1|CALML5|CBY1|...,ABCB8|AHNAK2|ALOX15B|ANKRD11|AP3D1|ATP12A|BACH...,AARS2|ABCA7|ABCB8|ABCC5|ABCF3|ABHD10|ACADVL|AC...
18,E2F6::SP1,SCC,A431,other|AND|other|XOR|OR|AND|other|other|other|OR,1|10|2|3|4|5|6|7|8|9,10,AND,E2F6,SP1,GDSD6|SCC,240,274,1385,ABCB6|ACTG1|ACTN4|ADAM15|AHNAK2|ALDH16A1|AP2S1...,ABCB6|ACTG1|ACTN4|ADAM15|AHNAK2|AKR1B10|ALDH16...,AARS2|ABCA7|ABCB6|ABCB8|ABCC5|ABCF3|ABHD10|ABL...
25,EGR1::SP3,SCC,A431,other|other|other|AND|LEADING_A|AND|other|OR|o...,1|10|2|3|4|5|6|7|8|9,10,AND,EGR1,SP3,GDSD6|SCC,281,327,1432,AAMP|ABCA7|ABCB6|ABCB8|ABCF3|ACTG1|ACTN4|ADAM1...,AAMP|ABCA7|ABCB6|ABCB8|ABCF3|ACTG1|ACTN4|ADAM1...,AAMP|AARS2|ABCA7|ABCB6|ABCB8|ABCC5|ABCF3|ABHD1...


In [36]:
new_mpra_df.to_csv(os.path.join(save_dir, 'target_gene_overlap_normal_cancer.csv'))

# 2. get genomic instances for above

In [8]:
tissue_mapper = {
    'Colon':'Colon', 'GDSD6':'GDSD6', 'GM12878':'GM12878', 'Melanocytes':'Melanocytes','Prostate':'Prostate',
    'SCC':'A431-CTRLi', 'MM':'COLO_SCR_DMSO'
}
# tissue_mapper = {
#     'Colon':['Colon'], 'GDSD6':['GDSD6'], 'GM12878':['GM12878'], 'Melanocytes':['Melanocytes'],
#     'SCC':['A431-CTRLi','CAL27-CTRLi','SCC13-CTRLi'], 'MM':['COLO_SCR_DMSO','WM_SCR_DMSO']
# }

file_pro_pro_mapper = {
    'GDSD6':'../data/processed/fig4_modelling/tf_tf_pairs/expr_GDSD6_pro_pro_vocab_info.csv',
    'Melanocytes':'../data/processed/fig4_modelling/tf_tf_pairs/expr_Melanocytes_pro_pro_vocab_info.csv',
    'Prostate':'../data/processed/fig4_modelling/tf_tf_pairs/expr_Prostate_pro_pro_vocab_info.csv',
    'SCC':'../data/processed/fig4_modelling/tf_tf_pairs_scc/expr_A431-CTRLi_pro_pro_vocab_info.csv',
    'MM':'../data/processed/fig4_modelling/tf_tf_pairs_mm/expr_COLO-SCR-DMSO_pro_pro_vocab_info.csv',
}

file_loop_loop_mapper = {
    'GDSD6':'../data/processed/fig4_modelling/tf_tf_pairs/expr_GDSD6_loop_loop_vocab_info.csv',
    'Melanocytes':'../data/processed/fig4_modelling/tf_tf_pairs/expr_Melanocytes_loop_loop_vocab_info.csv',
    'Prostate':'../data/processed/fig4_modelling/tf_tf_pairs/expr_Prostate_loop_loop_vocab_info.csv',
    'SCC':'../data/processed/fig4_modelling/tf_tf_pairs_scc/expr_A431-CTRLi_loop_loop_vocab_info.csv',
    'MM':'../data/processed/fig4_modelling/tf_tf_pairs_mm/expr_COLO-SCR-DMSO_loop_loop_vocab_info.csv',
}


In [141]:
def make_loop_counter(loop_df):
    loop_loop_dict = defaultdict(list)
    for anc_a,anc_b in zip(loop_df.source,loop_df.target):
        loop_loop_dict[anc_a].append(anc_b) 
        loop_loop_dict[anc_b].append(anc_a)
    return loop_loop_dict

def get_genomic_instances_pro_loop_single_gene(vocab, tissue, gene,
                                               anc_prom,loop_df,anc_foot,
                                               verbose=False):
    vocab_check = dict(zip(vocab.split('::'),vocab.split('::')[::-1]))

    if verbose:
        print('=======================')
        print( vocab, tissue)
        print(gene)
    
    # get anchors
    anc_prom_sel = anc_prom[anc_prom.name==gene]
    if verbose:
        print('anc_prom', anc_prom_sel.shape)
    if anc_prom_sel.shape[0]==0:
        if verbose:
            print(gene,'NO INSTANCES FOUND - no anchor promoters')
        return None
    
    # get anchors that loop to promoter anchors
    anc_prom_arr = anc_prom_sel.thickEnd.unique()
    loop_df_sel = loop_df[loop_df.source.isin(anc_prom_arr)|loop_df.target.isin(anc_prom_arr)]
    loop_loop_dict = make_loop_counter(loop_df_sel)
#     for anc in loop_loop_dict:
#         print(anc, len(loop_loop_dict[anc]))
    all_anc = sorted(set(anc_prom.thickEnd.to_list() +loop_df_sel.source.to_list() + loop_df_sel.target.to_list() ))
    anc_loop_arr = sorted(set(all_anc) - set(anc_prom_arr))
    if verbose:
        print('num all anchors', len(all_anc))#all_anc)
        print('num promoter anchors', len(anc_prom_arr))
        print('num looped anchors', len(anc_loop_arr))
  
    # get footprinting anchor data
    anc_foot_sel = anc_foot[anc_foot.tf.isin(vocab_check.keys())]
    anc_foot_pro = anc_foot_sel[anc_foot_sel.name.isin(anc_prom_arr)]
    anc_prom_arr_to_idx = dict(zip(anc_prom_arr, ['pro_'+str(idx) for idx in range(len(anc_prom_arr))]))
    anc_foot_pro['type']= anc_foot_pro['name'].map(anc_prom_arr_to_idx)
    if anc_foot_pro.shape[0]==0:
        if verbose:
            print(gene, 'NO INSTANCES FOUND - no promoter footprints')
        return None

    anc_foot_loop = pd.DataFrame()
    for idx, row  in anc_foot_pro[['name','tf','type']].drop_duplicates().iterrows():
        anc_pro = row['name']
        anc_pro_tf = row.tf
        anc_loop_tf = vocab_check[anc_pro_tf]
        anc_loop_arr_test = loop_loop_dict[anc_pro]
        anc_foot1 = anc_foot_sel[anc_foot_sel.name.isin(anc_loop_arr_test) & (anc_foot.tf==anc_loop_tf)]
        anc_foot1['type']= row.type +'+loop'
        anc_foot_loop = pd.concat([anc_foot_loop, anc_foot1])

    if anc_foot_loop.shape[0]==0:
        if verbose:
            print(gene, 'NO INSTANCES FOUND - no looped footprints')
        return None
        
    if verbose:
        print('number of promoter footprinted anchors with vocab',anc_foot_pro.shape)
        print('number of looped footprinted anchors with vocab',anc_foot_pro.shape)

    anc_foot_pro['name'] = gene
    genomic_instances = pd.concat([anc_foot_pro,anc_foot_loop])
    genomic_instances.columns = ['chr', 'start', 'stop', 'TSS', 'chr_m', 'start_m', 'stop_m', 'id_trim',
       'score', 'strand', 'qual','tf','type']
    genomic_instances.drop('qual',axis=1,inplace=True)
    genomic_instances['vocab'] = vocab
    genomic_instances['tissue'] = tissue
    genomic_instances['config'] = 'pro_loop'
#     genomic_instances['gene'] = gene
    
    if verbose:
        print('*** genomic instances per gene',genomic_instances.shape)
#         display(genomic_instances)
    return genomic_instances

def get_genomic_instances_pro_loop(vocab, tissue, list_genes=None,verbose=False, 
                          tf_id_to_name_dict=tf_id_to_name_dict,
                              save_dir=None):
    # get genes with this vocab (any configuraton)
    if verbose:
        print('=======================')
        print( vocab, tissue)
        print(list_genes)
        print(len(list_genes))
    # getting files
    anc_prom = pybedtools.BedTool('../data/interim/annon/promoter_anchors/promoter_'+tissue+'_annon.bed').to_dataframe()
    loop_df = pd.read_csv('../data/interim/merged/loops/'+tissue+'.loops.csv',index_col=0)
    anc_foot = pybedtools.BedTool('../data/interim/annon/anchor_footprinting/'+tissue+'_annon.bed').to_dataframe()
    anc_foot['tf'] = anc_foot.thickEnd.map(tf_id_to_name_dict)

    genomic_instances = pd.DataFrame()
    for gene in list_genes:
        genomic_instances1 = get_genomic_instances_pro_loop_single_gene(vocab, tissue, gene,anc_prom,loop_df,anc_foot,verbose=verbose)
        if genomic_instances1 is not None:
#             genomic_instances1['gene'] = gene
            print(gene, 'pro-loop',genomic_instances1.shape)
            if save_dir is not None:
                genomic_instances1.to_csv()
            genomic_instances = pd.concat([genomic_instances, genomic_instances1])
    
    if verbose:
        print('*** genomic instances',genomic_instances.shape)
        display(genomic_instances)
    if save_dir is not None:
        genomic_instances.to_csv(os.path.join(save_dir, '_'.join(vocab.split('::'))+'_genomic_instances.csv'))
    return genomic_instances


# def get_genomic_instances_pro_loop(vocab, tissue, list_genes=None,verbose=False, 
#                           tf_id_to_name_dict=tf_id_to_name_dict,
#                           save_dir=None):
#     # get genes with this vocab (any configuraton)
#     if verbose:
#         print('=======================')
#         print( vocab, tissue)
#         print(list_genes)
#         print(len(list_genes))
    
#     # get anchors
#     anc_prom = pybedtools.BedTool('../data/interim/annon/promoter_anchors/promoter_'+tissue+'_annon.bed').to_dataframe()
#     if list_genes is not None:
#         anc_prom = anc_prom[anc_prom.name.isin(list_genes)]
#     if verbose:
#         print('anc_prom', anc_prom.shape)
    
#     # get anchors that loop to promoter anchors
#     anc_prom_arr = anc_prom.thickEnd.unique()
#     if verbose:
#         print('num promoter anchors', len(anc_prom_arr))
    
#     # get loops
#     loop_df = pd.read_csv('../data/interim/merged/loops/'+tissue+'.loops.csv',index_col=0)
#     loop_df = loop_df[loop_df.source.isin(anc_prom_arr)|loop_df.target.isin(anc_prom_arr)]
#     all_anc = sorted(set(anc_prom.thickEnd.to_list() +loop_df.source.to_list() + loop_df.target.to_list() ))
#     if verbose:
#         print('num all anchors', len(all_anc))#all_anc)
#         display(loop_df)
    
#     # get footprinting anchor data
#     anc_foot = pybedtools.BedTool('../data/interim/annon/anchor_footprinting/'+tissue+'_annon.bed').to_dataframe()
#     anc_foot['tf'] = anc_foot.thickEnd.map(tf_id_to_name_dict)
#     anc_foot = anc_foot[anc_foot.tf.isin(vocab.split('::'))]
#     anc_foot = anc_foot[anc_foot.name.isin(all_anc)]
#     if verbose:
#         print('number of footprinted anchors with vocab',anc_foot.shape)
    
#     # get promoter footprint data
#     pro_anc_foot = pd.DataFrame()
#     pro_anc_foot_df = pybedtools.BedTool('../data/interim/annon/promoter_footprinting/promoter_'+tissue+'_annon.bed').to_dataframe()
#     pro_anc_foot_df['tf'] = pro_anc_foot_df.thickEnd.map(tf_id_to_name_dict)
#     if list_genes is not None:
#         # add the footprint info if it is in a targetted gene or if the tf is in a looped anchor region of interest
#         for gene in list_genes:
#             pro_anc_foot1 = pro_anc_foot_df[(pro_anc_foot_df.name==gene)|(pro_anc_foot_df.tf.isin(vocab.split('::')))]# not perfect will catch later???
            
#             pro_anc_foot1['anchor'] = pro_anc_foot_df.score + "_" + pro_anc_foot_df.strand.map(str)+"_"+ pro_anc_foot_df.thickStart.map(str)
#             pro_anc_foot1 = pro_anc_foot1[pro_anc_foot1.anchor.isin(all_anc)]
#             pro_anc_foot1.drop(columns=['anchor'],inplace=True)
#             pro_anc_foot = pd.concat([pro_anc_foot, pro_anc_foot1])
#     else:
#         pro_anc_foot = pro_anc_foot_df[pro_anc_foot_df.tf.isin(vocab.split('::'))]
    
#     if pro_anc_foot.shape[0]==0:
#         genomic_instances = pd.DataFrame(columns=['chr', 'start', 'stop', 'TSS', 'chr_m', 'start_m', 'stop_m', 'id_trim',
#        'score', 'strand', 'tf'])
#     else:
#         genomic_instances = pd.concat([anc_foot,pro_anc_foot])
#         genomic_instances.columns = ['chr', 'start', 'stop', 'TSS', 'chr_m', 'start_m', 'stop_m', 'id_trim',
#            'score', 'strand', 'qual','tf']
#         genomic_instances.drop('qual',axis=1,inplace=True)
#         genomic_instances['vocab'] = vocab
#         genomic_instances['tissue'] = tissue
#         genomic_instances['config'] = 'pro_loop'
    
#     if verbose:
#         print('*** genomic instances',genomic_instances.shape)
#         display(genomic_instances)
#     if save_dir is not None:
#         genomic_instances.to_csv(os.path.join(save_dir, '_'.join(vocab.split('::'))+'_genomic_instances.csv'))
#     return genomic_instances


def get_genomic_instances_all(vocab, tissue, list_genes,verbose=False, 
                          tf_id_to_name_dict=tf_id_to_name_dict,
                          save_prefix=None,rewrite=True):
    #pro-pro
    print('(((())))')
    print(vocab, tissue, len(list_genes))
    if (not rewrite) and save_prefix is not None:
        save_file = save_prefix + 'genomic_instances_'+'_'.join(vocab.split('::'))+"_"+tissue+'.csv'
        if os.path.exists(save_file):
            print('loading from file')
            genomic_instances_all = pd.read_csv(save_file,index_col=0)
            print('overall instances',genomic_instances_all.shape)
            return genomic_instances_all


    print('(((())))')
    print(vocab, tissue, len(list_genes))
    pro_pro_file = file_pro_pro_mapper[tissue]
    genomic_instances_pro_pro = pd.read_csv(pro_pro_file,index_col=0)
    genomic_instances_pro_pro = genomic_instances_pro_pro[genomic_instances_pro_pro.TSS.isin(list_genes)]
    genomic_instances_pro_pro = genomic_instances_pro_pro[genomic_instances_pro_pro.vocab==vocab]
    genomic_instances_pro_pro.drop('genes',axis=1,inplace=True)
    genomic_instances_pro_pro['config'] = 'pro_pro'
    genomic_instances_pro_pro['type']  = '-'
    print('pro-pro instances',genomic_instances_pro_pro.shape)
    
    # pro-loop

    genomic_instances_pro_loop = get_genomic_instances_pro_loop(vocab, tissue_mapper[tissue], list_genes,verbose=verbose, 
                          tf_id_to_name_dict=tf_id_to_name_dict,
                          save_dir=None)
    print('pro-loop instances',genomic_instances_pro_loop.shape)
    
    #loop-loop
    loop_loop_file = file_loop_loop_mapper[tissue]
    genomic_instances_loop_loop = pd.read_csv(loop_loop_file,index_col=0)
    genomic_instances_loop_loop = genomic_instances_loop_loop[genomic_instances_loop_loop.TSS.isin(list_genes)]
    genomic_instances_loop_loop = genomic_instances_loop_loop[genomic_instances_loop_loop.vocab==vocab]
    genomic_instances_loop_loop.drop('genes',axis=1,inplace=True)
    genomic_instances_loop_loop['config'] = 'loop_loop'
    genomic_instances_loop_loop['type']  = '-'
    print('loop-loop instances',genomic_instances_loop_loop.shape)

    genomic_instances_all = pd.concat([genomic_instances_pro_pro, genomic_instances_pro_loop, genomic_instances_loop_loop],ignore_index=True)
    print('overall instances',genomic_instances_all.shape)
    if save_prefix is not None:
        genomic_instances_all.to_csv(save_prefix + 'genomic_instances_'+'_'.join(vocab.split('::'))+"_"+tissue+'.csv')

    return genomic_instances_all

def get_target_genes(vocab,tissue, verbose=False):
    df_all_instances = get_genomic_instances_pro_loop(vocab, tissue,verbose=verbose)
    df = df_all_instances[~df_all_instances.TSS.str.startswith('chr')]
    print(len(df.TSS.unique()), 'target genes found for', vocab, 'in tissue: ', tissue)
    print(sorted(df.TSS.unique()))
    return df_all_instances

In [10]:
# %%time
# for idx, row in new_mpra_df.iterrows():
#     for tissue in row.tissues_to_check.split('|'):
#         get_genomic_instances_all(row.vocab, tissue, row.overlap_genes.split('|'),
#                                   verbose=False, 
#                                   save_prefix=save_dir)

# 3 Different vocabs same gene

 I was more interested in the overlap between the different vocabs but targeting the same gene in their respective tissues, for example if there is overlap between ARNT::SP1 target genes in SCC with BACH1::KLF5 target genes in GDSD6. Essentially wanting to get at cancer coopts a new vocab to regulate the same target genes that are regulated in normal by a different vocab.
 
 
 update 7/23/21
- do this for all skin type vocabs

In [128]:
# get mpra vocabs + configs
vocab_df = pd.read_csv('../data/processed/mpra/results_df_piv_match_count.csv',index_col=0)
# filter for skin stuff
vocab_df = vocab_df[vocab_df.tissue.isin(['GDSD6','SCC','Melanocytes','MM'])]

# get target genes for each AND vocab word in the correct tissue
vocab_to_target_genes = {}
for idx, row in vocab_df.iterrows():
    tf1,tf2 = row['vocab'].split('::')
    geneset_file = os.path.join('../data/processed/fig7_vocab_go/vocab_genesets/',row['tissue']+'_'+tf1 + '-' + tf2+'.txt' )
    geneset = set(pd.read_csv(geneset_file,header=None).iloc[:,0].values)
    vocab_to_target_genes[row.vocab] = geneset
#     print(row.vocab, len(geneset))

    
# filter for AND VOCABs
vocab_df = vocab_df[vocab_df.consensus_class=='AND']

# get vocab words for each tissue
GDSD6_AND_vocabs = vocab_df[vocab_df.tissue=='GDSD6'].vocab.values
SCC_AND_vocabs = vocab_df[vocab_df.tissue=='SCC'].vocab.values
Melanocytes_AND_vocabs = vocab_df[vocab_df.tissue=='Melanocytes'].vocab.values
MM_AND_vocabs = vocab_df[vocab_df.tissue=='MM'].vocab.values
AND_vocabs = [GDSD6_AND_vocabs,SCC_AND_vocabs, Melanocytes_AND_vocabs, MM_AND_vocabs ]
for v_list in AND_vocabs:
    print(len(v_list))

10
10
5
10


In [167]:
intersect_genes_MC_MM = pd.DataFrame(index=Melanocytes_AND_vocabs, columns = MM_AND_vocabs)
for MC_vocab in Melanocytes_AND_vocabs:
    for MM_vocab in MM_AND_vocabs:
        intersect_genes = vocab_to_target_genes[MC_vocab].intersection(vocab_to_target_genes[MM_vocab])
        intersect_genes_MC_MM.at[MC_vocab, MM_vocab] = '|'.join(sorted(intersect_genes))
intersect_genes_MC_MM_long = intersect_genes_MC_MM.reset_index().melt('index').drop_duplicates()
intersect_genes_MC_MM_long.columns = ['MC_vocab','MM_vocab','intersect_genes']
intersect_genes_MC_MM_long['num_genes'] = intersect_genes_MC_MM_long.intersect_genes.apply(lambda str_v: len(str_v.split('|')))
intersect_genes_MC_MM_long.to_csv(os.path.join(save_dir, 'intersect_genes_MC_MM_long.csv'))
# intersect_genes_MC_MM_long # size 5x9 = 45

In [171]:
intersect_genes_KC_SCC = pd.DataFrame(index=GDSD6_AND_vocabs, columns = SCC_AND_vocabs)
for KC_vocab in GDSD6_AND_vocabs:
    for SCC_vocab in SCC_AND_vocabs:
        intersect_genes = vocab_to_target_genes[KC_vocab].intersection(vocab_to_target_genes[SCC_vocab])
        intersect_genes_KC_SCC.at[KC_vocab, SCC_vocab] = '|'.join(sorted(intersect_genes))
intersect_genes_KC_SCC_long = intersect_genes_KC_SCC.reset_index().melt('index').drop_duplicates()
intersect_genes_KC_SCC_long.columns = ['KC_vocab','SCC_vocab','intersect_genes']
intersect_genes_KC_SCC_long['num_genes'] = intersect_genes_KC_SCC_long.intersect_genes.apply(lambda str_v: len(str_v.split('|')))
intersect_genes_KC_SCC_long.to_csv(os.path.join(save_dir, 'intersect_genes_KC_SCC_long.csv'))
intersect_genes_KC_SCC_long #size 9x9 = 81

,KC_vocab,SCC_vocab,intersect_genes,num_genes
0,BACH1::KLF5,ARNT::SP1,ABCB8|AHNAK2|ANKRD11|AP3D1|CALML5|CELSR1|CHRNB...,69
1,EGR2::FOSL2,ARNT::SP1,ABCB8|ABCF3|ABHD10|ACTR3|ADAM15|AHNAK|AHNAK2|A...,150
2,EGR2::RARG,ARNT::SP1,ABCB8|ABCF3|ABHD10|ACADVL|AHNAK2|AP1S1|AP2S1|A...,126
3,EGR2::RORA,ARNT::SP1,ABCA7|ABCB6|ABCF3|ACADVL|ACTR3|ADAM15|AHNAK2|A...,137
4,HMGA1::KLF5,ARNT::SP1,AAMP|ABCA7|ABCB8|ABHD10|ACADVL|ACTG1|ADAMTS1|A...,160
...,...,...,...,...
76,HMGA1::KLF5,SP1::ZBTB17,AAMP|ABCA7|ABCB8|ABHD10|ACADVL|ACTG1|ACTN4|ADA...,232
77,KLF5::MAFG,SP1::ZBTB17,ABCB8|ACTN4|ADAM15|ADAMTS1|AHNAK2|AKR1B1|ANKRD...,133
78,KLF5::MAFK,SP1::ZBTB17,ABCB6|ABCB8|ACADVL|ADAM15|ADAMTS1|AHNAK2|AKR1B...,120
79,MAF::RORA,SP1::ZBTB17,ACADVL|ACTN4|AHNAK2|ANKZF1|AP3D1|ARHGDIA|ATG9A...,95


In [181]:
%%time
# takes awhile
for idx, row in intersect_genes_MC_MM_long.iterrows():
    print(idx, '+++++++++++++++')
    get_genomic_instances_all(row.MC_vocab, 'Melanocytes', row.intersect_genes.split('|'),
                              verbose=False, 
                              save_prefix='../data/processed/fig7_vocab_go/genomic_instance_overlap/'+str(idx)+'_')
    get_genomic_instances_all(row.MM_vocab, 'MM', row.intersect_genes.split('|'),
                              verbose=False, 
                              save_prefix='../data/processed/fig7_vocab_go/genomic_instance_overlap/'+str(idx)+'_')

0 +++++++++++++++
(((())))
ETV2::PRRX1 Melanocytes 29
pro-pro instances (42, 14)
pro-loop instances (257, 14)
loop-loop instances (284, 14)
overall instances (583, 14)
(((())))
EGR2::NFE2L1 MM 29
pro-pro instances (0, 14)
pro-loop instances (223, 14)
loop-loop instances (0, 14)
overall instances (223, 14)
1 +++++++++++++++
(((())))
FOXD3::PRRX1 Melanocytes 16
pro-pro instances (138, 14)
pro-loop instances (214, 14)
loop-loop instances (248, 14)
overall instances (600, 14)
(((())))
EGR2::NFE2L1 MM 16
pro-pro instances (0, 14)
pro-loop instances (114, 14)
loop-loop instances (0, 14)
overall instances (114, 14)
2 +++++++++++++++
(((())))
KLF15::ZNF589 Melanocytes 70
pro-pro instances (210, 14)
pro-loop instances (483, 14)
loop-loop instances (674, 14)
overall instances (1367, 14)
(((())))
EGR2::NFE2L1 MM 70
pro-pro instances (0, 14)
pro-loop instances (505, 14)
loop-loop instances (0, 14)
overall instances (505, 14)
3 +++++++++++++++
(((())))
KLF5::NFYB Melanocytes 22
pro-pro instances (0

In [182]:
%%time
# takes awhile
for idx, row in intersect_genes_KC_SCC_long.iterrows():
    print(idx, '+++++++++++++++')
    get_genomic_instances_all(row.KC_vocab, 'GDSD6', row.intersect_genes.split('|'),
                              verbose=False, 
                              save_prefix='../data/processed/fig7_vocab_go/genomic_instance_overlap/'+str(idx)+'_')
    get_genomic_instances_all(row.SCC_vocab, 'SCC', row.intersect_genes.split('|'),
                              verbose=False, 
                              save_prefix='../data/processed/fig7_vocab_go/genomic_instance_overlap/'+str(idx)+'_')
    

0 +++++++++++++++
(((())))
BACH1::KLF5 GDSD6 69
pro-pro instances (0, 14)
pro-loop instances (313, 14)
loop-loop instances (134, 14)
overall instances (447, 14)
(((())))
ARNT::SP1 SCC 69
pro-pro instances (0, 14)
pro-loop instances (1644, 14)
loop-loop instances (0, 14)
overall instances (1644, 14)
1 +++++++++++++++
(((())))
EGR2::FOSL2 GDSD6 150
pro-pro instances (14, 14)
pro-loop instances (1051, 14)
loop-loop instances (558, 14)
overall instances (1623, 14)
(((())))
ARNT::SP1 SCC 150
pro-pro instances (0, 14)
pro-loop instances (2943, 14)
loop-loop instances (0, 14)
overall instances (2943, 14)
2 +++++++++++++++
(((())))
EGR2::RARG GDSD6 126
pro-pro instances (12, 14)
pro-loop instances (873, 14)
loop-loop instances (486, 14)
overall instances (1371, 14)
(((())))
ARNT::SP1 SCC 126
pro-pro instances (0, 14)
pro-loop instances (2760, 14)
loop-loop instances (0, 14)
overall instances (2760, 14)
3 +++++++++++++++
(((())))
EGR2::RORA GDSD6 137
pro-pro instances (120, 14)
pro-loop instanc

# update 7/24/2021 - see if we can see how many genomic instances we can get for each vocab word in the normal and cancer form 

use folder `genomic_instance_normal_cancer_comp`



In [46]:
# takes awhile
genomic_instance_comp_df = pd.DataFrame()
genomic_instance_comp_df['vocab'] = ''
genomic_instance_comp_df['tissues_to_check'] = ''
genomic_instance_comp_df['num_normal_instances'] = 0
genomic_instance_comp_df['num_cancer_instances'] = 0
for idx, row in new_mpra_df.iterrows():
    # get row
    print(row['vocab'],row.tissue)
    tf1,tf2 = row['vocab'].split('::')
    genomic_instance_comp_df.at[idx,'vocab'] = row['vocab']
    
    # get tissues to check
    if row.tissue in ['GDSD6',  'SCC']:
        tissues_to_check = ['GDSD6',  'SCC']
    else:
        tissues_to_check = ['Melanocytes','MM']
    genomic_instance_comp_df.at[idx, 'tissues_to_check'] = '|'.join(tissues_to_check )
  
    # get genomic instances
    num_instances = [0,0] # for normal, and cancer
    
    for i_mpra,mpra_tissue in enumerate(tissues_to_check):
        geneset_file = os.path.join(vocab_geneset_dir,row['tissue']+'_'+tf1 + '-' + tf2+'_'+mpra_tissue+'.txt' )
        if os.path.exists(geneset_file):
            geneset = set(pd.read_csv(geneset_file,header=None).iloc[:,0].values)
            if len(geneset)>0:
                instances = get_genomic_instances_all(row.vocab, mpra_tissue, sorted(geneset),
                                          verbose=False, rewrite=False,
                                          save_prefix='../data/processed/fig7_vocab_go/genomic_instance_normal_cancer_comp/'+str(idx)+'_')
                num_instances[i_mpra] = len(instances)
    genomic_instance_comp_df.at[idx,'num_normal_instances'] = num_instances[0]
    genomic_instance_comp_df.at[idx,'num_cancer_instances'] = num_instances[1]
    print(genomic_instance_comp_df.loc[idx,:])


ARID5B::LEF1 MM
(((())))
ARID5B::LEF1 Melanocytes 13
loading from file
overall instances (4024, 14)
(((())))
ARID5B::LEF1 MM 464
loading from file
overall instances (574639, 14)
vocab                     ARID5B::LEF1
tissues_to_check        Melanocytes|MM
num_normal_instances              4024
num_cancer_instances            574639
Name: 1, dtype: object
ARNT::SP1 SCC
(((())))
ARNT::SP1 GDSD6 91
loading from file
overall instances (104879, 14)
(((())))
ARNT::SP1 SCC 828
loading from file
overall instances (897199, 14)
vocab                   ARNT::SP1
tissues_to_check        GDSD6|SCC
num_normal_instances       104879
num_cancer_instances       897199
Name: 5, dtype: object
BACH1::KLF5 GDSD6
(((())))
BACH1::KLF5 GDSD6 100
loading from file
overall instances (58058, 14)
(((())))
BACH1::KLF5 SCC 988
loading from file
overall instances (572001, 14)
vocab                   BACH1::KLF5
tissues_to_check          GDSD6|SCC
num_normal_instances          58058
num_cancer_instances         57200

In [47]:
genomic_instance_comp_df.to_csv(os.path.join(save_dir, 'genomic_instance_comp_df.csv'))
genomic_instance_comp_df


,vocab,tissues_to_check,num_normal_instances,num_cancer_instances
1,ARID5B::LEF1,Melanocytes|MM,4024.0,574639.0
5,ARNT::SP1,GDSD6|SCC,104879.0,897199.0
8,BACH1::KLF5,GDSD6|SCC,58058.0,572001.0
18,E2F6::SP1,GDSD6|SCC,360936.0,1776878.0
25,EGR1::SP3,GDSD6|SCC,319286.0,1334645.0
26,EGR2::FOSL2,GDSD6|SCC,260414.0,1184169.0
31,EGR2::NFE2L1,Melanocytes|MM,0.0,565239.0
32,EGR2::RARG,GDSD6|SCC,185825.0,1081153.0
33,EGR2::RORA,GDSD6|SCC,190733.0,0.0
43,EN2::STAT1,Melanocytes|MM,455.0,204327.0


# misc stuff
Can you see if chr22:38572526 is in our TF footprint data for MC or any of the melanoma cell lines?



In [9]:
mc_foot_bed = []
for file in glob.glob('../data/interim/merged/footprinting/*'):
    if os.path.basename(file).startswith('COLO'):
        mc_foot_bed.append(file)
    elif os.path.basename(file).startswith('WM_'):
        mc_foot_bed.append(file)
    elif os.path.basename(file).startswith('SKMEL'):
        mc_foot_bed.append(file)
    elif os.path.basename(file).startswith('Melanoc'):
        mc_foot_bed.append(file)
mc_foot_bed        

['../data/interim/merged/footprinting/COLO_SCR_PLX_merged.bed',
 '../data/interim/merged/footprinting/COLO_shMITF_PLX_merged.bed',
 '../data/interim/merged/footprinting/COLO_shMITF_DMSO_merged.bed',
 '../data/interim/merged/footprinting/WM_SCR_DMSO_merged.bed',
 '../data/interim/merged/footprinting/Melanocytes_merged.bed',
 '../data/interim/merged/footprinting/SKMEL5_SCR_DMSO_merged.bed',
 '../data/interim/merged/footprinting/COLO_SCR_DMSO_merged.bed']

In [10]:
results_df = pd.DataFrame()
for foot_bed in mc_foot_bed:
    print(os.path.basename(foot_bed))
    foot_df = pybedtools.BedTool(foot_bed).to_dataframe()
    foot_df = foot_df[(foot_df.chrom=='chr22')]
    foot_df = foot_df[(foot_df.start<=38572526)&(foot_df.end>=38572526)]
    foot_df['tissue'] = os.path.basename(foot_bed).split('_merged')[0]
    results_df = pd.concat([results_df, foot_df])
    display(foot_df)

COLO_SCR_PLX_merged.bed


,chrom,start,end,name,score,strand,thickStart,tissue


COLO_shMITF_PLX_merged.bed


,chrom,start,end,name,score,strand,thickStart,tissue


COLO_shMITF_DMSO_merged.bed


KeyboardInterrupt: 

# update 06/24/2021
Also, since we redid the Fig 3 TF expression/GOTerms figure POU2F2 is no longer a good vignette since it doesn't come up as an enriched motif in KCs :/, can you get me the GDSD6 and Prostate anchor and loop instances of ATF4, SMAD4, and CEBPG? Esp if any of those have FLG as a target gene (GDSD6) or UGDH as a target gene (prostate)


In [203]:
gds_flg_df = pd.DataFrame()
for tf1 in [ 'ATF4', 'SMAD4',  'CEBPG']:
    for tf2 in [ 'ATF4', 'SMAD4',  'CEBPG']:
        vocab = tf1+'::'+tf2
        df =get_genomic_instances_all(vocab, 'GDSD6', ['FLG'])
        gds_flg_df = pd.concat([gds_flg_df, df])

(((())))
ATF4::ATF4 GDSD6 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
ATF4::SMAD4 GDSD6 1
pro-pro instances (0, 14)
pro-loop instances (1, 14)
loop-loop instances (0, 14)
overall instances (1, 14)
(((())))
ATF4::CEBPG GDSD6 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
SMAD4::ATF4 GDSD6 1
pro-pro instances (0, 14)
pro-loop instances (1, 14)
loop-loop instances (0, 14)
overall instances (1, 14)
(((())))
SMAD4::SMAD4 GDSD6 1
pro-pro instances (0, 14)
pro-loop instances (1, 14)
loop-loop instances (0, 14)
overall instances (1, 14)
(((())))
SMAD4::CEBPG GDSD6 1
pro-pro instances (0, 14)
pro-loop instances (1, 14)
loop-loop instances (0, 14)
overall instances (1, 14)
(((())))
CEBPG::ATF4 GDSD6 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
CEBPG::SMAD4 GDSD6 1
pro-pro instances

In [204]:
gds_flg_df

,chr,start,stop,TSS,chr_m,start_m,stop_m,id_trim,score,strand,tf,vocab,tissue,config
0,chr1,152415000,152420000,chr1_152415000_152420000,chr1,152415595,152415606,SMAD4_HUMAN.H11MO.0.B.pwm.trim,10.83232,+,SMAD4,ATF4::SMAD4,GDSD6,pro_loop
0,chr1,152415000,152420000,chr1_152415000_152420000,chr1,152415595,152415606,SMAD4_HUMAN.H11MO.0.B.pwm.trim,10.83232,+,SMAD4,SMAD4::ATF4,GDSD6,pro_loop
0,chr1,152415000,152420000,chr1_152415000_152420000,chr1,152415595,152415606,SMAD4_HUMAN.H11MO.0.B.pwm.trim,10.83232,+,SMAD4,SMAD4::SMAD4,GDSD6,pro_loop
0,chr1,152415000,152420000,chr1_152415000_152420000,chr1,152415595,152415606,SMAD4_HUMAN.H11MO.0.B.pwm.trim,10.83232,+,SMAD4,SMAD4::CEBPG,GDSD6,pro_loop
0,chr1,152415000,152420000,chr1_152415000_152420000,chr1,152415595,152415606,SMAD4_HUMAN.H11MO.0.B.pwm.trim,10.83232,+,SMAD4,CEBPG::SMAD4,GDSD6,pro_loop


In [207]:
pros_ugdh_df = pd.DataFrame()
for tf1 in [ 'ATF4', 'SMAD4',  'CEBPG']:
    for tf2 in [ 'ATF4', 'SMAD4',  'CEBPG']:
        vocab = tf1+'::'+tf2
        df =get_genomic_instances_all(vocab, 'Prostate', ['UGDH'])
        pros_ugdh_df = pd.concat([pros_ugdh_df, df])

(((())))
ATF4::ATF4 Prostate 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
ATF4::SMAD4 Prostate 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
ATF4::CEBPG Prostate 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
SMAD4::ATF4 Prostate 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
SMAD4::SMAD4 Prostate 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
SMAD4::CEBPG Prostate 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
CEBPG::ATF4 Prostate 1
pro-pro instances (0, 14)
pro-loop instances (0, 14)
loop-loop instances (0, 14)
overall instances (0, 14)
(((())))
CEBPG::SMAD4 Pros

In [208]:
pros_ugdh_df

,chr,start,stop,TSS,chr_m,start_m,stop_m,id_trim,score,strand,tf,vocab,tissue,config


Then can I also get the list of SMAD4 footprints in anchors/loops and the target genes identified in Prostate? I can try to find a good gene candidate to show

In [236]:
smad4_pros = get_target_genes('SMAD4', 'Prostate')

25 target genes found for SMAD4 in tissue:  Prostate
['AC013449.1', 'ARID3B', 'BMF', 'C10orf55', 'CLCNKA', 'EGFL8', 'FAM114A1', 'GAL3ST1', 'HIST2H3A', 'HSPB7', 'ICAM1', 'KIRREL2', 'LMTK3', 'MGAT4B', 'MMP14', 'NBPF24', 'RP11-293M10.1', 'RP11-474G23.1', 'RUNX3', 'S1PR4', 'SLC4A9', 'SQSTM1', 'TLX2', 'TSC22D4', 'TUSC2']


In [233]:
smad4_pros.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/SMAD4_prostate_manual.csv')

In [234]:
smad4_pros.start_m - smad4_pros.stop#.TSS.unique().shape

465       -68
8813    -4221
8909    -2202
11977   -2637
14158   -4483
         ... 
39492   -2190
40354    -637
40535   -1201
41312   -1259
43324    -316
Length: 288, dtype: int64

# Can I get the gene lists for TF footprints of ATF4 and CEBPG in KCD6 and Prostate pro-loops? 




In [12]:
atf4_kcd6 = get_target_genes('ATF4', 'GDSD6')
atf4_pros = get_target_genes('ATF4', 'Prostate')

38 target genes found for ATF4 in tissue:  GDSD6
['AL160175.1', 'ARHGDIA', 'ATAT1', 'C4BPB', 'CCDC9', 'CCL17', 'CSTA', 'DYNC1LI2', 'FAM114A1', 'GDF15', 'GPR87', 'GSS', 'HAPLN2', 'ID1', 'IGFBP5', 'IL36G', 'ING2', 'KIAA0754', 'LSM10', 'MYH15', 'NFATC4', 'PARP2', 'PDP1', 'PIP5KL1', 'PRDM10', 'PYGB', 'RAET1E', 'REN', 'RHOH', 'RNASE7', 'RP11-762I7.5', 'S100A6', 'SESN2', 'SH3RF2', 'SLC25A14', 'SLC35A4', 'SLC9A1', 'STRA6']
13 target genes found for ATF4 in tissue:  Prostate
['ATAT1', 'CTU1', 'DYNC1LI2', 'GDF15', 'LRRC27', 'MRPS21', 'NDUFA4L2', 'OTOP1', 'PARP2', 'PKIG', 'RP11-529K1.3', 'STK32C', 'STRA6']


In [15]:
cebpg_kcd6 = get_target_genes('CEBPG', 'GDSD6')
cebpg_pros = get_target_genes('CEBPG', 'Prostate')

44 target genes found for CEBPG in tissue:  GDSD6
['AL160175.1', 'ANKRD11', 'ARHGDIA', 'ATAT1', 'C12orf45', 'C4BPB', 'CCDC9', 'CCL17', 'CSTA', 'CYGB', 'DYNC1LI2', 'FAM114A1', 'GDF15', 'GPR87', 'GSS', 'HAPLN2', 'ID1', 'IGFBP5', 'IL36G', 'ING2', 'KIAA0754', 'KRT33A', 'LSM10', 'NFATC4', 'PARP2', 'PDP1', 'PIP5KL1', 'PRDM10', 'PYGB', 'RAET1E', 'REN', 'RHOH', 'RNASE7', 'RP11-468E2.2', 'RP11-762I7.5', 'S100A6', 'SERPINB13', 'SESN2', 'SH3RF2', 'SLC25A14', 'SLC35A4', 'SLC9A1', 'STRA6', 'TOP1']
14 target genes found for CEBPG in tissue:  Prostate
['ATAT1', 'CTU1', 'CYGB', 'DYNC1LI2', 'GDF15', 'LRRC27', 'MRPS21', 'NDUFA4L2', 'NFIX', 'PARP2', 'PKIG', 'RP11-529K1.3', 'STK32C', 'STRA6']


In [16]:
atf4_kcd6.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/ATF4_kcd6_manual.csv')
atf4_pros.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/ATF4_prostate_manual.csv')
cebpg_kcd6.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/CEBPG_kcd6_manual.csv')
cebpg_pros.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/CEBPG_prostate_manual.csv')


In [17]:
cebpg_kcd6

,chr,start,stop,TSS,chr_m,start_m,stop_m,id_trim,score,strand,tf,vocab,tissue,config
29,chr1,845000,850000,chr1_845000_850000,chr1,846718,846728,CEBPG_HUMAN.H11MO.0.B.pwm.trim,12.215445,+,CEBPG,CEBPG,GDSD6,pro_loop
3662,chr1,4760000,4765000,chr1_4760000_4765000,chr1,4763236,4763246,CEBPG_HUMAN.H11MO.0.B.pwm.trim,10.263728,+,CEBPG,CEBPG,GDSD6,pro_loop
6574,chr1,8270000,8275000,chr1_8270000_8275000,chr1,8272237,8272247,CEBPG_HUMAN.H11MO.0.B.pwm.trim,15.931376,+,CEBPG,CEBPG,GDSD6,pro_loop
18248,chr1,17515000,17520000,chr1_17515000_17520000,chr1,17519080,17519090,CEBPG_HUMAN.H11MO.0.B.pwm.trim,13.381499,+,CEBPG,CEBPG,GDSD6,pro_loop
19855,chr1,19325000,19330000,chr1_19325000_19330000,chr1,19325089,19325099,CEBPG_HUMAN.H11MO.0.B.pwm.trim,12.599845,+,CEBPG,CEBPG,GDSD6,pro_loop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69604,chr6,30582619,30585119,ATAT1,chr6,30582811,30582821,CEBPG_HUMAN.H11MO.0.B.pwm.trim,13.230063,+,CEBPG,CEBPG,GDSD6,pro_loop
72029,chr6,150207238,150209738,RAET1E,chr6,150208740,150208750,CEBPG_HUMAN.H11MO.0.B.pwm.trim,12.604143,+,CEBPG,CEBPG,GDSD6,pro_loop
76362,chr8,94858035,94860535,PDP1,chr8,94858942,94858952,CEBPG_HUMAN.H11MO.0.B.pwm.trim,11.196498,-,CEBPG,CEBPG,GDSD6,pro_loop
78485,chr9,130681076,130683576,PIP5KL1,chr9,130681864,130681874,CEBPG_HUMAN.H11MO.0.B.pwm.trim,10.880806,-,CEBPG,CEBPG,GDSD6,pro_loop



# update 07/15/2021
Can you send me the KLF5 TF footprint motif loci + target genes for colon and KC?

Also maybe PATZ1 colon vs renal



In [12]:
KLF5_kcd6 = get_target_genes('KLF5', 'GDSD6')
KLF5_colon = get_target_genes('KLF5', 'Colon')

343 target genes found for KLF5 in tissue:  GDSD6
['ABCA7', 'ABCF3', 'AC003101.1', 'AC009041.2', 'AC018512.1', 'AC024580.1', 'AC068533.7', 'AC104841.2', 'AC105020.1', 'ACLY', 'ACPT', 'ACTN4', 'ACVR1B', 'ACVRL1', 'ADAM15', 'ADM2', 'ADRA1B', 'AHNAK2', 'AL118506.1', 'AL136376.1', 'AL136531.1', 'ALKBH7', 'AMT', 'ANGPTL2', 'ANKHD1', 'ANKHD1-EIF4EBP3', 'ANKK1', 'ANKZF1', 'ANXA8L1', 'AOC2', 'AP000349.1', 'AP2S1', 'ARAP1', 'ARFRP1', 'ARHGEF12', 'ARMC12', 'ARSG', 'ATG9A', 'ATN1', 'ATP12A', 'ATP5B', 'BAI2', 'BANF2', 'BHLHE41', 'BIRC3', 'C10orf55', 'C11orf84', 'C11orf91', 'C17orf61-PLSCR3', 'C19orf45', 'C1orf134', 'C1orf210', 'C1orf216', 'C1orf56', 'C20orf112', 'C22orf15', 'C2CD4D', 'C4A', 'C6orf223', 'C9orf114', 'C9orf173', 'C9orf69', 'CA11', 'CALML5', 'CARD9', 'CCDC120', 'CCDC51', 'CCDC69', 'CCDC78', 'CCNI2', 'CCT3', 'CDC42SE1', 'CELSR2', 'CEMP1', 'CES5A', 'CHMP2A', 'CHMP4C', 'CISH', 'CLDN6', 'CLK3', 'CNIH2', 'CNTFR', 'COL6A1', 'COPZ2', 'COX7A1', 'CRTC2', 'CRYAB', 'CTD-2267D19.3', 'CTDSP1', 'CT

In [13]:
PATZ1_colon = get_target_genes('PATZ1', 'Colon')
PATZ1_renal = get_target_genes('PATZ1', 'Renal')

558 target genes found for PATZ1 in tissue:  Colon
['ABCA7', 'AC009041.2', 'AC010327.2', 'AC010646.3', 'AC068533.7', 'AC104841.2', 'AC105020.1', 'AC127496.1', 'AC132216.1', 'ACADVL', 'ACLY', 'ACSL1', 'ACSS2', 'ACTN4', 'ACTR3C', 'ACVRL1', 'ADAM15', 'ADAP1', 'ADM', 'ADM5', 'ADRA1B', 'AFAP1', 'AGXT', 'AHNAK', 'AHNAK2', 'AKT1S1', 'AL031320.1', 'AL136376.1', 'AL136531.1', 'AL589739.1', 'AL645608.1', 'AMOTL2', 'AMT', 'ANKRD66', 'ANKZF1', 'ANXA2', 'AOC2', 'AP000349.1', 'AP000769.1', 'AP1G2', 'APH1A', 'APOBEC3C', 'APOBEC3D', 'ARAP1', 'ARFRP1', 'ARHGDIA', 'ARHGEF12', 'ARHGEF28', 'ARL14', 'ARMC12', 'ATG9A', 'ATN1', 'ATP2A2', 'ATP2B3', 'ATP5B', 'ATP5H', 'B3GAT3', 'B4GALNT1', 'B4GALT3', 'B9D2', 'BANF2', 'BCO2', 'BHLHE41', 'BMF', 'BOLA2B', 'BRMS1L', 'BTBD18', 'C11orf91', 'C16orf3', 'C17orf96', 'C19orf24', 'C19orf43', 'C19orf54', 'C1QB', 'C1QTNF6', 'C1orf100', 'C1orf116', 'C1orf134', 'C1orf172', 'C1orf216', 'C1orf85', 'C20orf112', 'C22orf15', 'C22orf26', 'C22orf29', 'C22orf34', 'C2CD4D', 'C3orf52', 

In [15]:
KLF5_kcd6.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/KLF5_kcd6_manual.csv')
KLF5_colon.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/KLF5_colon_manual.csv')
PATZ1_colon.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/PATZ1_colon_manual.csv')
PATZ1_renal.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/PATZ1_renal_manual.csv')


# update 10/13/2021

Also, can you find the genomic instances (chr location) of ARNT+SP1 and the TF footprint nucleotides associated with then in A431s looped to or in the promoter of the CYP1B1 gene?



In [ ]:
vocab = 'ARNT::SP1'
df =get_genomic_instances_all(vocab, 'SCC', ['CYP1B1'])


In [75]:
df.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/ARNT::SP1_a431_instances_all.csv')


In [134]:
# tissue='A431-CTRLi'
# list_genes = ['CYP1B1']
# anc_prom = pybedtools.BedTool('../data/interim/annon/promoter_anchors/promoter_'+tissue+'_annon.bed').to_dataframe()
# anc_prom = anc_prom[anc_prom.name.isin(list_genes)]
# anc_prom_arr = anc_prom.thickEnd.unique()
    
# loop_df = pd.read_csv('../data/interim/merged/loops/'+tissue+'.loops.csv',index_col=0)
# loop_df = loop_df[loop_df.source.isin(anc_prom_arr)|loop_df.target.isin(anc_prom_arr)]
# all_anc = sorted(set(anc_prom.thickEnd.to_list() +loop_df.source.to_list() + loop_df.target.to_list() ))

# anc_foot = pybedtools.BedTool('../data/interim/annon/anchor_footprinting/'+tissue+'_annon.bed').to_dataframe()
# anc_foot['tf'] = anc_foot.thickEnd.map(tf_id_to_name_dict)
# anc_foot = anc_foot[anc_foot.tf.isin(vocab.split('::'))]
# anc_foot = anc_foot[anc_foot.name.isin(all_anc)]


# pro_anc_foot_df = pybedtools.BedTool('../data/interim/annon/promoter_footprinting/promoter_'+tissue+'_annon.bed').to_dataframe()
# pro_anc_foot_df['tf'] = pro_anc_foot_df.thickEnd.map(tf_id_to_name_dict)
# pro_anc_foot = pd.DataFrame()

# for gene in list_genes:
#     pro_anc_foot1 = pro_anc_foot_df[(pro_anc_foot_df.name==gene)|(pro_anc_foot_df.tf.isin(vocab.split('::')))]# not perfect will catch later???
#     pro_anc_foot1['anchor'] = pro_anc_foot_df.score + "_" + pro_anc_foot_df.strand.map(str)+"_"+ pro_anc_foot_df.thickStart.map(str)
#     pro_anc_foot1 = pro_anc_foot1[pro_anc_foot1.anchor.isin(all_anc)]
#     pro_anc_foot1.drop(columns=['anchor'],inplace=True)
#     pro_anc_foot = pd.concat([pro_anc_foot, pro_anc_foot1])

In [88]:
vocab

'ARNT::SP1'

In [98]:
tissue = 'A431-CTRLi'
anc_prom = pybedtools.BedTool('../data/interim/annon/promoter_anchors/promoter_'+tissue+'_annon.bed').to_dataframe()
loop_df = pd.read_csv('../data/interim/merged/loops/'+tissue+'.loops.csv',index_col=0)
anc_foot = pybedtools.BedTool('../data/interim/annon/anchor_footprinting/'+tissue+'_annon.bed').to_dataframe()
anc_foot['tf'] = anc_foot.thickEnd.map(tf_id_to_name_dict)



So ARNT-SP1, SP1-ZBTB14, and SP1-ZNF770 genomic loci + associated target genes in SCC would be great (edited) 

Laura Donohue  11 minutes ago
Second tier would be EGR2-NFE2L1, NFE2L1-NR3C1, and SOX10-SOX13 in Melanoma
Third tier is EGR2-FOSL2, EGR2-RORA, HMGA1-KLF5, KLF5-MAFK, and PRDM1-RORA in KC

In [140]:
%%time
# vocab = 'ARNT::SP1'
for vocab in ['ARNT::SP1','SP1::ZBTB14','SP1::ZNF770']:
    list_genes = vocab_to_target_genes[vocab]
    df =get_genomic_instances_all(vocab, 'SCC', list_genes)
    df.to_csv('../data/processed/fig7_vocab_go/genomic_instance_overlap/'+vocab+'_a431_instances_all.csv')
    

(((())))
ARNT::SP1 SCC 828
(((())))
ARNT::SP1 SCC 828
pro-pro instances (59, 15)


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWar

pro-loop instances (2237, 15)
loop-loop instances (0, 15)
overall instances (2296, 15)
(((())))
SP1::ZBTB14 SCC 1018
(((())))
SP1::ZBTB14 SCC 1018


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:224: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



pro-pro instances (376, 15)
pro-loop instances (3546, 15)
loop-loop instances (59, 15)
overall instances (3981, 15)
(((())))
SP1::ZNF770 SCC 1523
(((())))
SP1::ZNF770 SCC 1523
pro-pro instances (1215, 15)
pro-loop instances (15836, 15)
loop-loop instances (99, 15)
overall instances (17150, 15)
CPU times: user 3h 5min 50s, sys: 3min 4s, total: 3h 8min 55s
Wall time: 1h 7min 41s


In [121]:
# # gene = 'ABCA7'
# for gene in ['AARS2', 'ABCA7',  'ACBD4', 'ACER2', 'ACLY', 'ACTG1', 'ACTN4', 'ADAP1', 'ADM', 'ADM5', 'ADRA1B', 'AHNAK', 'AHNAK2', 'AL109659.1', 'AL118506.1', 'AL136376.1', 'AL136531.1', 'AL590714.1', 'ALDH3B1', 'ANKRD9', 'ANKZF1', 'ANO9', 'AOC2', 'AP000349.1', 'AP3D1', 'APH1A', 'APOC1', 'ARAP1', 'ARHGEF12', 'ARHGEF4', 'ARRB1', 'ARSG', 'ASAP3', 'ASS1', 'ATAD3B', 'ATG9A', 'ATP11C', 'ATP2A2', 'ATP2C2', 'ATP5B', 'B4GALT3', 'B9D2', 'BIRC3', 'BMF', 'BTBD18', 'C12orf36', 'C16orf3', 'C17orf100', 'C17orf96', 'C19orf24', 'C19orf55', 'C1QTNF6', 'C1orf200', 'C1orf216', 'C22orf15', 'C2CD4D', 'C4A', 'C5orf55', 'C6orf1', 'C8orf82', 'C9orf114', 'C9orf16', 'C9orf173', 'C9orf50', 'C9orf69', 'C9orf78', 'CACNG6', 'CARD10', 'CARD9', 'CASP16', 'CCDC120', 'CCDC24', 'CCDC51', 'CCDC78', 'CD52', 'CDC20', 'CDC42SE1', 'CDK4', 'CDT1', 'CEMP1', 'CHAF1A', 'CHCHD1', 'CHMP2A', 'CIRBP', 'CISH', 'CLDN7', 'CLK3', 'CNIH2', 'CNTD2', 'COA7', 'COL9A3', 'COMMD6', 'CRIP1', 'CRTC2', 'CTB-96E2.2', 'CTD-2521M24.10', 'CTNND1', 'CTRC', 'CUEDC2', 'CUL9', 'CYB561D1', 'CYB561D2', 'CYHR1', 'DAZAP1', 'DCAKD', 'DDB1', 'DDX5', 'DEGS2', 'DIEXF', 'DISP2', 'DKC1', 'DLG3', 'DNAJB1', 'DNAJC28', 'DNAJC5G', 'DNPEP', 'DPM2', 'DPP9', 'DRAXIN', 'DSCR3', 'DVL1', 'DYNC1LI2', 'E2F1', 'EFCAB4B', 'EFNA3', 'EFNA4', 'EHF', 'EIF3H', 'EMD', 'EML3', 'ENTPD8', 'ERCC1', 'EXOC3L1', 'EXOC3L4', 'FAM129C', 'FAM175B', 'FAM189B', 'FAM213B', 'FAM83F', 'FAU', 'FBXL16', 'FES', 'FKBP10', 'FKBP2', 'FKSG48', 'FLII', 'FOSL1', 'FRMD1', 'FTL', 'FTSJ1', 'FTSJ2', 'FUT6', 'G6PC2', 'GABARAPL2', 'GABPB2', 'GEMIN2', 'GFER', 'GJA4', 'GJC2', 'GKAP1', 'GNB5', 'GPATCH4', 'GPHA2', 'GPR107', 'GSDMA', 'GSDMD', 'GSTM2', 'HAGHL', 'HIST1H1D', 'HIST1H2AD', 'HIST1H2BF', 'HIST1H3D', 'HLCS', 'HNRNPH1', 'HOGA1', 'HOXB8', 'HOXC10', 'HOXD1', 'HPN', 'HSD17B10', 'HSPB6', 'ICAM1', 'ID1', 'IDH1', 'IDH3B', 'IER5L', 'IFI30', 'IFITM5', 'IGIP', 'IKZF4', 'INF2', 'INMT-FAM188B', 'IZUMO1', 'JAK1', 'JPH2', 'KBTBD4', 'KCNJ9', 'KIAA0895L', 'KIFAP3', 'KIFC2', 'KLHL21', 'KMT2E', 'KRI1', 'LAMTOR1', 'LEPREL4', 'LGI3', 'LLGL2', 'LMF1', 'LMTK3', 'LPAR2', 'LRRC25', 'LRRC46', 'LRRC71', 'LSM10', 'LTBR', 'LUZP1', 'LY6G6C', 'LY6G6D', 'LY6G6E', 'MAD2L2', 'MAF1', 'MAK16', 'MAPKAPK3', 'MARCH9', 'MB', 'MED31', 'MEGF8', 'MESP2', 'MGAT1', 'MGAT4B', 'MIOX', 'MLLT11', 'MMP25', 'MOCS1', 'MPDU1', 'MPHOSPH9', 'MR1', 'MROH1', 'MRPL10', 'MRPL45', 'MRPL49', 'MSL1', 'MT3', 'MYADML2', 'MYL4', 'N4BP1', 'NANOS3', 'NCR3', 'NECAB3', 'NEFH', 'NFIX', 'NHLRC4', 'NHSL1', 'NISCH', 'NLGN2', 'NOL3', 'NOXO1', 'NPB', 'NPRL2', 'NR2C2AP', 'NRN1L', 'NTMT1', 'NUBP2', 'NUDT1', 'NUPR1L', 'NXNL1', 'NYAP1', 'ODF3L1', 'OGFR', 'OPN1MW', 'OPN1MW2', 'OSBPL2', 'OSR1', 'OTUB1', 'OXTR', 'PABPN1L', 'PACSIN3', 'PAGR1', 'PAQR7', 'PCDHGA1', 'PCSK4', 'PHF13', 'PI4K2A', 'PIANP', 'PIDD', 'PIGQ', 'PIKFYVE', 'PIM3', 'PLCB3', 'PLCD3', 'PLEKHB2', 'PMPCA', 'PNLDC1', 'POU2F2', 'POU5F1', 'PPIL2', 'PPP1R16A', 'PPP4C', 'PPP6R1', 'PRAP1', 'PRCC', 'PRDX2', 'PRMT5', 'PROC', 'PRPF6', 'PRSS33', 'PRSS36', 'PSMB3', 'PSMC6', 'PSMD8', 'PSRC1', 'PTCD2', 'PTRH1', 'QPCTL', 'RAB17', 'RAB3A', 'RABEP2', 'RBAK-RBAKDN', 'REC8', 'REEP4', 'REEP6', 'RGL2', 'RHCG', 'RINT1', 'RIPK1', 'RNASE13', 'RNF175', 'RNF224', 'ROM1', 'ROPN1B', 'RP1-130H16.18', 'RP1-4G17.5', 'RP11-293M10.1', 'RP11-342M21.2', 'RP11-345J4.3', 'RP11-347C12.3', 'RP11-51F16.8', 'RP11-834C11.12', 'RP13-512J5.1', 'RPL10', 'RPL22', 'RPL24', 'RPL3', 'RPLP2', 'RUSC1', 'S100A11', 'SALL2', 'SAMD10', 'SCAMP3', 'SCN4A', 'SCNM1', 'SDCCAG3', 'SDR39U1', 'SEPT1', 'SERPINB10', 'SERPINF2', 'SERTAD3', 'SIRT4', 'SIRT7', 'SIX1', 'SLC10A5', 'SLC16A3', 'SLC22A18AS', 'SLC23A3', 'SLC25A41', 'SLC30A2', 'SLC30A3', 'SLC50A1', 'SLC5A2', 'SLC6A11', 'SLC9A5', 'SLC9A6', 'SMCO1', 'SMTNL1', 'SNRPD1', 'SNRPD2', 'SOX8', 'SP2', 'SPATA24', 'SPSB3', 'SPTBN2', 'SQSTM1', 'SRA1', 'SRPK3', 'SRSF2', 'SSBP3-AS1', 'SSC5D', 'SSRP1', 'SSSCA1', 'ST3GAL2', 'STARD10', 'STMN1', 'STMN3', 'SYNGAP1', 'SYNGR3', 'TAB1', 'TAOK2', 'TBX10', 'TEX28', 'TGFB1I1', 'TGFBR3L', 'TGM2', 'THAP3', 'TINF2', 'TKTL1', 'TLCD1', 'TLX2', 'TMA7', 'TMC4', 'TMEM145', 'TMEM206', 'TMEM52', 'TMEM63A', 'TMEM88', 'TMIE', 'TMPO', 'TNFAIP2', 'TNFAIP8L2', 'TNFRSF4', 'TPD52L2', 'TRAF7', 'TRIM36', 'TRPM2', 'TTC39A', 'TTC6', 'TTLL10', 'TUBB1', 'TUBB3', 'UBA7', 'UBXN11', 'UCHL3', 'USP20', 'UTP23', 'VGF', 'VPS11', 'WDR24', 'WDR64', 'WDR74', 'WNT3', 'WNT9B', 'XIRP1', 'XXYLT1', 'XXbac-BPG32J3.20', 'XXcos-LUCA11.5', 'ZFHX3', 'ZIC2', 'ZMYND19', 'ZNF444', 'ZNF513', 'ZNF629', 'ZNF821']:
#     get_genomic_instances_pro_loop_single_gene(vocab, tissue, gene,anc_prom, loop_df, anc_foot,
#                                          verbose=True)

ARNT::SP1 A431-CTRLi
AARS2
anc_prom (2, 9)
num all anchors 28601
num promoter anchors 2
num looped anchors 28599
NO INSTANCES FOUND - no promoter footprints
ARNT::SP1 A431-CTRLi
ABCA7
anc_prom (3, 9)
num all anchors 28616
num promoter anchors 3
num looped anchors 28613


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


chr19_1025000_1030000 34


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


NO INSTANCES FOUND - no looped footprints
ARNT::SP1 A431-CTRLi
ACBD4
anc_prom (3, 9)
num all anchors 28649
num promoter anchors 3
num looped anchors 28646
chr17_43200000_43205000 81
NO INSTANCES FOUND - no looped footprints
ARNT::SP1 A431-CTRLi
ACER2
anc_prom (2, 9)
num all anchors 28620
num promoter anchors 2
num looped anchors 28618
chr9_19395000_19400000 20
NO INSTANCES FOUND - no looped footprints
ARNT::SP1 A431-CTRLi
ACLY
anc_prom (3, 9)
num all anchors 28614
num promoter anchors 3
num looped anchors 28611
chr17_40070000_40075000 13
chr17_40075000_40080000 32
NO INSTANCES FOUND - no looped footprints
ARNT::SP1 A431-CTRLi
ACTG1
anc_prom (3, 9)
num all anchors 28703
num promoter anchors 3
num looped anchors 28700
chr17_79480000_79485000 164


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


chr17_79485000_79490000 10
number of promoter footprinted anchors with vocab (10, 13)
number of looped footprinted anchors with vocab (10, 13)
*** genomic instances (15, 15)


,chr,start,stop,TSS,chr_m,start_m,stop_m,id_trim,score,strand,tf,type,vocab,tissue,config
1025338,chr17,79480000,79485000,chr17_79480000_79485000,chr17,79480116,79480137,SP1_HUMAN.H11MO.0.A.pwm.trim,14.500829,-,SP1,pro_1,ARNT::SP1,A431-CTRLi,pro_loop
1025346,chr17,79480000,79485000,chr17_79480000_79485000,chr17,79480126,79480136,SP1_HUMAN.H11MO.1.A.pwm.trim,13.382828,-,SP1,pro_1,ARNT::SP1,A431-CTRLi,pro_loop
1025365,chr17,79480000,79485000,chr17_79480000_79485000,chr17,79480189,79480199,SP1_HUMAN.H11MO.1.A.pwm.trim,18.026685,+,SP1,pro_1,ARNT::SP1,A431-CTRLi,pro_loop
1025374,chr17,79480000,79485000,chr17_79480000_79485000,chr17,79480193,79480214,SP1_HUMAN.H11MO.0.A.pwm.trim,7.210513,+,SP1,pro_1,ARNT::SP1,A431-CTRLi,pro_loop
1025393,chr17,79480000,79485000,chr17_79480000_79485000,chr17,79480579,79480600,SP1_HUMAN.H11MO.0.A.pwm.trim,10.309607,-,SP1,pro_1,ARNT::SP1,A431-CTRLi,pro_loop
1025404,chr17,79480000,79485000,chr17_79480000_79485000,chr17,79480589,79480599,SP1_HUMAN.H11MO.1.A.pwm.trim,14.347540,-,SP1,pro_1,ARNT::SP1,A431-CTRLi,pro_loop
1025523,chr17,79480000,79485000,chr17_79480000_79485000,chr17,79482054,79482064,SP1_HUMAN.H11MO.1.A.pwm.trim,11.402480,+,SP1,pro_1,ARNT::SP1,A431-CTRLi,pro_loop
1025556,chr17,79485000,79490000,chr17_79485000_79490000,chr17,79486071,79486081,SP1_HUMAN.H11MO.1.A.pwm.trim,13.620915,-,SP1,pro_2,ARNT::SP1,A431-CTRLi,pro_loop
1025587,chr17,79485000,79490000,chr17_79485000_79490000,chr17,79486628,79486649,SP1_HUMAN.H11MO.0.A.pwm.trim,6.760533,-,SP1,pro_2,ARNT::SP1,A431-CTRLi,pro_loop
1025616,chr17,79485000,79490000,chr17_79485000_79490000,chr17,79486648,79486658,SP1_HUMAN.H11MO.1.A.pwm.trim,15.451729,+,SP1,pro_2,ARNT::SP1,A431-CTRLi,pro_loop


ARNT::SP1 A431-CTRLi
ACTN4
anc_prom (3, 9)
num all anchors 28624
num promoter anchors 3
num looped anchors 28621
chr19_39125000_39130000 39
chr19_39130000_39135000 17
NO INSTANCES FOUND - no looped footprints
ARNT::SP1 A431-CTRLi
ADAP1
anc_prom (3, 9)
num all anchors 28628
num promoter anchors 3
num looped anchors 28625
chr7_985000_990000 48
chr7_990000_995000 3


KeyboardInterrupt: 